In [79]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [80]:
# data read 
words = open('/home/risal/X3s4c5/Makemore/names.txt','r').read().splitlines()

In [81]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)

In [82]:
# build the dataset
block_size = 3
def build_dataset(words):
    X, Y= [] , []
    
    for w in words:
        context  = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X,Y

import random
random.seed(42)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

X_trn, Y_trn = build_dataset(words[:n1])
X_dev, Y_dev = build_dataset(words[n1:n2])
X_tst, Y_tst = build_dataset(words[n2:])

In [83]:
# defining utility fucntion, used to compare manual backpropagation
def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [84]:
n_emb = 10
n_hidden = 200

# MLP components 
g = torch.Generator().manual_seed(123)
C = torch.randn((vocab_size,n_emb), generator = g)
# Linear 1
W1 = torch.randn((n_emb * block_size, n_hidden), generator = g) * (5/3)/((n_emb * block_size)**0.5)
b1 = torch.randn(n_hidden, generator = g)  * 0
# Linear 2
W2 = torch.randn((n_hidden, vocab_size), generator = g) * 0.1
b2 = torch.randn(vocab_size, generator = g ) * 0
# batchNorm parameters 
bngain = torch.ones((1, n_hidden))
bnbias = torch.zeros((1, n_hidden))

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

12297


In [85]:
batch_size = 32
n= batch_size
#  Construct mini batch
ix = torch.randint(0,X_trn.shape[0], (batch_size,), generator = g)
Xb, Yb = X_trn[ix], Y_trn[ix]

In [86]:
emb = C[Xb]
embcat = emb.view(emb.shape[0],-1)
# linear layer 1
hprebn = embcat @ W1 + b1 
# BatchNorm layer
bnmeani = 1 /n * hprebn.sum(0, keepdim =True) #hprebn.mean(0, keepdim = True) 
bndiff = hprebn - bnmeani    #(Xi - mean)
bndiff2 = bndiff**2          
bnvar = 1 / (n-1) * (bndiff2).sum(0, keepdim = True)  #Bessel's correction sampled by (n-1) 
bnvar_inv = (bnvar + 1e-5) ** -0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity 
h = torch.tanh(hpreact)
# linear layer 2 
logits = h @ W2 + b2
# Loss calculation(calculate loglikelihood)
logit_maxes = logits.max(1, keepdim = True).values
norm_logits = logits - logit_maxes
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdim = True)
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# Pytorch backward pass 
for p in parameters: 
    p.grad = None
for t in [logprobs, probs, counts_sum_inv, counts_sum, counts, norm_logits, logit_maxes, logits, h, hpreact,
         bnraw, bnvar_inv, bnvar, bndiff2, bndiff, bnmeani,hprebn, embcat, emb]:
    t.retain_grad()
loss.backward()
loss

tensor(3.7188, grad_fn=<NegBackward0>)

# Backprop

In [87]:
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = - 1.0/ n 
dprobs = (1 / probs) * dlogprobs # Using chain rule (local derivative times the derivative of loss with respective to its output)
dcounts_sum_inv = (counts * dprobs).sum(1, keepdim =True)  # ?? (managing broadcasting of [32,27] to [32,1])
dcounts = counts_sum_inv * dprobs 
dcounts_sum = (-1*counts_sum**-2) * dcounts_sum_inv
dcounts += torch.ones_like(counts) * dcounts_sum  #
dnorm_logits = counts * dcounts
dlogit_maxes = (-1 * dnorm_logits).sum(1, keepdim = True)
dlogits = dnorm_logits.clone()
dlogits += F.one_hot(logits.max(1).indices, num_classes = logits.shape[1]) * dlogit_maxes
# Linear layer 2
dh = dlogits @ W2.T
dW2 = h.T @ dlogits
db2 = dlogits.sum(0) 
# Non-linearity
dhpreact = (1.0 - h**2) * dh    ### minor error 
# BatchNorm layer 
dbnbias = dhpreact.sum(0, keepdim = True)
dbngain = (bnraw * dhpreact).sum(0, keepdim = True)
dbnraw = bngain * dhpreact
dbndiff = bnvar_inv * dbnraw  ## has multiple branches 
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim = True)
dbnvar = (-0.5 * (bnvar + 1e-5) ** -1.5) * dbnvar_inv
dbndiff2 = (1 / (n-1) * torch.ones_like(bndiff2)) * dbnvar
dbndiff += (2 * bndiff) * dbndiff2 # another branch of dbndiff 
dbnmeani = (-1 * dbndiff).sum(0, keepdim = True)
dhprebn = dbndiff.clone()
dhprebn += (1 /n * torch.ones_like(hprebn)) * dbnmeani 
# Linear layer 1
dembcat = dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0)
demb = dembcat.view(emb.shape)
dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        ix = Xb[k, j]
        dC[ix] += demb[k,j ]

# Comparision

In [88]:
cmp('logprobs', dlogprobs, logprobs )
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('dnorm_logits', dnorm_logits, norm_logits)
cmp('dlogit_maxes', dlogit_maxes, logit_maxes)
cmp('dlogits', dlogits, logits)
cmp('dh', dh, h)
cmp('dW2', dW2, W2)
cmp('db2', db2, b2)
cmp('dhpreact', dhpreact, hpreact)
cmp('dbnbias',  dbnbias, bnbias)
cmp('dbngain', dbngain, bngain)
cmp('dbnraw', dbnraw, bnraw)
cmp('dbndiff', dbndiff, bndiff)
cmp('dbnvar_inv', dbnvar_inv, bnvar_inv)
cmp('dbnvar',dbnvar, bnvar )
cmp('dbndiff2', dbndiff2 , bndiff2)
cmp('dbnmeani', dbnmeani, bnmeani)
cmp('dhprebn', dhprebn, hprebn)
cmp('dembcat', dembcat, embcat)
cmp('dW1', dW1, W1)
cmp('db1', db1, b1)
cmp('demb', demb , emb)
cmp('dC', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
dnorm_logits    | exact: True  | approximate: True  | maxdiff: 0.0
dlogit_maxes    | exact: True  | approximate: True  | maxdiff: 0.0
dlogits         | exact: True  | approximate: True  | maxdiff: 0.0
dh              | exact: True  | approximate: True  | maxdiff: 0.0
dW2             | exact: True  | approximate: True  | maxdiff: 0.0
db2             | exact: True  | approximate: True  | maxdiff: 0.0
dhpreact        | exact: False | approximate: True  | maxdiff: 4.656612873077393e-10
dbnbias         | exact: False | approximate: True  | maxdiff: 7.450580596923828e-09
dbngain         | exact: False | approximate: True  | maxdiff: 1.862645149230957e-09
dbnraw  

# Backprop through actual chunk of mathmetical operations

## Through losses

In [89]:
# Forward pass
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:',(loss_fast - loss).item())

3.7187986373901367 diff: -2.384185791015625e-07


In [90]:
# Backward pass 
dlogits = F.softmax(logits, 1)
dlogits[range(n),Yb] -= 1
dlogits /= n
cmp('dlogits', dlogits, logits)

dlogits         | exact: False | approximate: True  | maxdiff: 7.450580596923828e-09


## Through BatchNorm

In [91]:
# Forward pass 
hpreact_fast = bngain * (hprebn - hprebn.mean(0, keepdim = True)) / torch.sqrt(hprebn.var(0, keepdim =True, unbiased=True))
print('max_diff:', (hpreact_fast - hpreact).abs().max())

max_diff: tensor(2.2411e-05, grad_fn=<MaxBackward1>)


In [92]:
# Calculate hprebn given dhpreact; might need some variable from the forward pass
dhprebn = bngain * bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0))
cmp('dhprebn', dhprebn, hprebn)

dhprebn         | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10


# Putting all together 

In [93]:
n_emb = 10
n_hidden = 200

# MLP components 
g = torch.Generator().manual_seed(123)
C = torch.randn((vocab_size,n_emb), generator = g)
# Linear 1
W1 = torch.randn((n_emb * block_size, n_hidden), generator = g) * (5/3)/((n_emb * block_size)**0.5)
b1 = torch.randn(n_hidden, generator = g)  * 0
# Linear 2
W2 = torch.randn((n_hidden, vocab_size), generator = g) * 0.1
b2 = torch.randn(vocab_size, generator = g ) * 0
# batchNorm parameters 
bngain = torch.ones((1, n_hidden))
bnbias = torch.zeros((1, n_hidden))

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = False

12297


In [94]:
# MLP
max_steps = 30000
batch_size = 32
losses_train = []
with torch.no_grad():
    for i in range(max_steps):
        
        # mini batch
        ix = torch.randint(0,X_trn.shape[0], (batch_size,), generator = g)
        Xb, Yb = X_trn[ix], Y_trn[ix]
        
        # forward passes 
        emb = C[Xb]
        embcat = emb.view(emb.shape[0],-1)
        # linear layer 1
        hprebn = embcat @ W1 + b1 
        # BatchNorm layer
        bnmeani = 1 /n * hprebn.sum(0, keepdim =True) #hprebn.mean(0, keepdim = True) 
        bndiff = hprebn - bnmeani    #(Xi - mean)
        bndiff2 = bndiff**2          
        bnvar = 1 / (n-1) * (bndiff2).sum(0, keepdim = True)  #Bessel's correction sampled by (n-1) 
        bnvar_inv = (bnvar + 1e-5) ** -0.5
        bnraw = bndiff * bnvar_inv
        hpreact = bngain * bnraw + bnbias
        # Non-linearity 
        h = torch.tanh(hpreact)
        # linear layer 2 
        logits = h @ W2 + b2
        loss = F.cross_entropy(logits,  Y_trn[ix]) 
    
        # backward pass 
        for p in parameters:
            p.grad = None
        # loss.backward()
        ## Code manual backpropagation here
        # Loss 
        dlogits = F.softmax(logits, 1)
        dlogits[range(n),Yb] -= 1
        dlogits /= n
        # Linear layer 2
        dh = dlogits @ W2.T
        dW2 = h.T @ dlogits
        db2 = dlogits.sum(0) 
                
        # Non-linearity
        dhpreact = (1.0 - h**2) * dh    
        # BatchNorm layer
        dbnbias = dhpreact.sum(0, keepdim = True)
        dbngain = (bnraw * dhpreact).sum(0, keepdim = True)
        dhprebn = bngain * bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0))
            
        # Linear layer 1
        dembcat = dhprebn @ W1.T
        dW1 = embcat.T @ dhprebn
        db1 = dhprebn.sum(0)
        demb = dembcat.view(emb.shape)
        dC = torch.zeros_like(C)
        for k in range(Xb.shape[0]):
            for j in range(Xb.shape[1]):
                ix = Xb[k, j]
                dC[ix] += demb[k,j ] 
        ### parameters = [C, W1, b1, W2, b2, bngain, bnbias]
        # dC, dW1, db1, dW2, db2, dbngain, dbnbias = None, None,None, None, None, None,None
        grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
        # update params
        lr = 0.1 
        for p,grad in zip(parameters, grads):
            # p.data += -lr * p.grad
            p.data += -lr * grad

        # track stats
        if i % 1000 == 0:
            print(f"{i:7d} / {max_steps:7d}: {loss.item():.4f}")
        losses_train.append(loss.item())

    # if i >= 100:
    #     break

      0 /   30000: 3.7188
   1000 /   30000: 2.2761
   2000 /   30000: 2.4004
   3000 /   30000: 2.4984
   4000 /   30000: 2.2252
   5000 /   30000: 2.0395
   6000 /   30000: 2.1538
   7000 /   30000: 2.2782
   8000 /   30000: 2.3779
   9000 /   30000: 2.3555
  10000 /   30000: 2.1661
  11000 /   30000: 2.5271
  12000 /   30000: 2.7327
  13000 /   30000: 2.4283
  14000 /   30000: 2.2728
  15000 /   30000: 1.9629
  16000 /   30000: 2.6917
  17000 /   30000: 2.3592
  18000 /   30000: 2.0944
  19000 /   30000: 2.3980
  20000 /   30000: 2.0161
  21000 /   30000: 2.1292
  22000 /   30000: 2.6313
  23000 /   30000: 2.0480
  24000 /   30000: 2.8023
  25000 /   30000: 1.9816
  26000 /   30000: 2.5508
  27000 /   30000: 2.2111
  28000 /   30000: 1.8077
  29000 /   30000: 2.1178


In [72]:
# Checking grad values 
# for p, g in  zip(parameters, grads):
#     cmp(str(tuple(p.shape)), g, p)

In [95]:
# 
with torch.no_grad():
    emb = C[X_trn]
    embcat = emb.view((emb.shape[0], -1))
    hpreact = embcat@ W1 + b1
    bnmean = hpreact.mean(0, keepdim = True)
    bnvar = hpreact.var(0, keepdim = True, unbiased = True)

In [96]:
@torch.no_grad()
def split_loss(split):
    X,Y = {
        'train': (X_trn, Y_trn),
        'val' : (X_dev, Y_dev),
        'test' : (X_tst, Y_tst),
    }[split]
    emb = C[X]
    embcat = emb.view((emb.shape[0], -1))
    hpreact = embcat @ W1 + b1
    hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)** -0.5 + bnbias # Batch norm formula and scaling and shifting  
    h = torch.tanh(hpreact)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits,  Y) 
    print(split, loss.item())

split_loss('train')
split_loss('val')

train 2.1506688594818115
val 2.4073147773742676


In [99]:
# Sampling from the model
g  = torch.Generator().manual_seed(123)
for i in range(1):
    out = []
    context = [0] * block_size
    while True:
        emb = C[torch.tensor([context])]
        embcat = emb.view((emb.shape[0], -1))
        h = torch.tanh(embcat @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim = 1)

        ix = torch.multinomial(probs, num_samples = 1, replacement = True, generator = g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    print(''.join(out))

agjznjjzzhvtjqwmwkzjrjgjjfuygfgjzjwjhrjdbazaqwfwfjlffzdguzkmhgqzzggkggehpgdgfkgggbgujghjqfzwcjujjjzgjgjzkccgjqqsbzjkgjzkjtjgjbcgkbvcxogwjsfkkjjjhhwjnzhgwjdcjjbgchfcghcjznjazjjrzkgbfjzcjmpggjgeczbyzjggjjjwmkdtglfgjnjzzggbghjghjgjcdjkzfzjkjjbcjymjdsbcwljjgtgdkqhgjyqyjjgzjjtgcjgkjjjgzzjjzcgjrcggffjhgzjgcgqjgggywffkdkjxgyjccfcwcwkajunujgfbbhzjimssbjlcjdshzjccjgyjjqbzgqgdfjgfhjszggcajcwjxjjwrgzjjrjjmrjjujcuzkhzsmjgcalzzmqzmbkjcjzjjpfbzcjzfjgpgzvjzbfbgmggsqjfgsazzjjgjjcaehjqjqkcgjzggfrgigfgjrzhsjzjrmqbjkfkbgrqkkjgjjkznzjgskjsgjugjobkffcffsxqdmfjfgzbbjkgfjzfjqhjqgjcbsjzjkqdagjopzqduzjdwgofjsjmxdwucgbggzdbkkjjglcvgwxkgjjgfffpzgqzjgjjgznfmrugwbfzccjhggzjybgcjhbdcgfgjgbgfdjskzvkrohjggzbgrmggblygjjcjkdjumhhjjjdkjjgrgggnkkxbjfkwdbjfkfzzjwfgxgjwjgjggbgbwvjffgbhgwfjjpbnckfjjfqmsgqjgzfhvffzjjjjfdbhzjgwfkgjjqbsbjcsjjptzjkjgkfgnsgdwntkujgkmbgzcwckzhzjvjznjzhczowgagjhdkgxugqfjcjjfjcjfzrjjhgfgzifjsfqghjjmmajcjbxgjmxzjygajjkcjjjzjjjmhsbgbqmkyzgjggnjdkgjjecjgjjjosgjqgjnjgbjjqfqzkhjggddjbgvggggkfvrghwgigffs